Selenay Buse Batıbay 30294

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
schema_pd = pd.read_csv("schema.csv")
freeform_df = pd.read_csv("freeformResponses.csv")
multichoice_df = pd.read_csv("multipleChoiceResponses.csv", encoding="latin1")
convrates_df = pd.read_csv("conversionRates.csv")

<ipython-input-13-3c57e4aa5c50>:2: DtypeWarning: Columns (5,17,21,38,50) have mixed types. Specify dtype option on import or set low_memory=False.
  freeform_df = pd.read_csv("freeformResponses.csv")
<ipython-input-13-3c57e4aa5c50>:3: DtypeWarning: Columns (31,83,86,87,98,99,109,116,123,124,127,129,130,164) have mixed types. Specify dtype option on import or set low_memory=False.
  multichoice_df = pd.read_csv("multipleChoiceResponses.csv", encoding="latin1")


Q2-a
P(A|B) WHERE A IS BEING A PROGRAMMER AND B IS USING C/C++

In [14]:
C_users_df = multichoice_df[multichoice_df['WorkToolsSelect'].str.contains('C\/C\+\+', na=False)]
total_c_users = C_users_df.shape[0]

programmers_c_users_df = C_users_df[C_users_df['CurrentJobTitleSelect'].str.contains('Programmer', na=False)]
prog_count = programmers_c_users_df.shape[0]

probability_programmer_given_c_cpp = 100 * prog_count / total_c_users

print("Probability that a respondent is currently employed as a Programmer given they use C/C++ at work = {:.2f}%".format(probability_programmer_given_c_cpp))



Probability that a respondent is currently employed as a Programmer given they use C/C++ at work = 2.23%


Q2-b

In [15]:
majored_df = multichoice_df[multichoice_df['MajorSelect'].str.contains('computer science|mathematics|statistics', case=False, na=False)]
math_cs_stat_majored_count = majored_df.shape[0]

data_scientists_in_majored_df = majored_df[majored_df['CurrentJobTitleSelect'].str.contains('Data Scientist', na=False)]
data_scientists_in_majored_df_count = data_scientists_in_majored_df.shape[0]

probability_CsMathStat_dataScientists = 100 *data_scientists_in_majored_df_count/math_cs_stat_majored_count

print("Probability that a respondent is a Data Scientist given they have majored incomputer science, mathematics or statistics = {:.2f}%".format(probability_CsMathStat_dataScientists))

Probability that a respondent is a Data Scientist given they have majored incomputer science, mathematics or statistics = 15.97%


Q2-c

In [16]:
annual_compensations_df = pd.DataFrame({
    'CompensationAmount': pd.to_numeric(multichoice_df['CompensationAmount'].str.replace(',', ''), errors='coerce'),
    'CompensationCurrency': multichoice_df['CompensationCurrency'],
    'EmployerIndustry': multichoice_df['EmployerIndustry']
}) #creating a new dataframe with the related elements in multichoice_df

annual_compensations_df.dropna(subset=['CompensationAmount', 'CompensationCurrency'], inplace=True)

annual_df = annual_compensations_df.merge(convrates_df, left_on='CompensationCurrency', right_on='originCountry', how='left')
annual_df['CompensationAmountinUSD'] = annual_df['CompensationAmount'] * annual_df['exchangeRate']

annual_compensation_bigger_than_40k_df = annual_df[annual_df['CompensationAmountinUSD'] > 40000]
bigger_than_40k_count = annual_compensation_bigger_than_40k_df.shape[0]

tech_workers_in_high_income_df = annual_compensation_bigger_than_40k_df[annual_compensation_bigger_than_40k_df['EmployerIndustry'].str.contains('Technology', na=False)]
tech_worker_count = tech_workers_in_high_income_df.shape[0]

probability = 100 * tech_worker_count/ bigger_than_40k_count

print("Probability that a respondent works in the Technology industry given that they earn more than 40,000 USD annually = {:.2f}%".format(probability))

Probability that a respondent works in the Technology industry given that they earn more than 40,000 USD annually = 19.35%


Q2-d

In [17]:
age_over_30_bachelor_df = multichoice_df[(multichoice_df['Age'] > 30) & (multichoice_df['FormalEducation'].isin(["Bachelor's degree", "Master's degree", "Doctoral degree", "Professional Degree"]))]
joint_probability = 100 *age_over_30_bachelor_df.shape[0] / multichoice_df.shape[0]

print("Joint probability of a respondent being over 30 years old and having a at least a Bachelor’s degree = {:.2f}%".format(joint_probability))

Joint probability of a respondent being over 30 years old and having a at least a Bachelor’s degree = 36.90%


Q2-e

In [9]:
data_scientist_df = multichoice_df[(multichoice_df['CurrentJobTitleSelect'] == 'Data Scientist') &
                                   (multichoice_df['MajorSelect'].isin(['Computer Science', 'Mathematics or statistics']))]

# Calculate the probability
probability = 100 * data_scientist_df.shape[0] / multichoice_df.shape[0]

print("Probability that a respondent is a Data Scientist who majored in Computer Science, Mathematics or statistics = {:.2f}%".format(probability))


Probability that a respondent is a Data Scientist who majored in Computer Science, Mathematics or statistics = 6.32%


Q2-f


In [10]:
new_df = pd.DataFrame({
    'CompensationAmount': pd.to_numeric(multichoice_df['CompensationAmount'].str.replace(',', ''), errors='coerce'),
    'CompensationCurrency': multichoice_df['CompensationCurrency'],
    'WorkMethodsFrequencyCross-Validation': multichoice_df['WorkMethodsFrequencyCross-Validation'],
    'Country': multichoice_df['Country']
})

new_df_for_france = new_df.merge(convrates_df, left_on='CompensationCurrency', right_on='originCountry', how='left')
new_df_for_france['CompensationAmountUSD'] = new_df_for_france['CompensationAmount'] * new_df_for_france['exchangeRate']
new_df_for_france.drop(['CompensationAmount', 'CompensationCurrency', 'originCountry', 'exchangeRate', 'Unnamed: 0'], axis=1, inplace=True)

new_df_for_france = new_df_for_france[
    (new_df_for_france['Country'] == 'France') &
    (new_df_for_france['CompensationAmountUSD'] < 100000) &
    (new_df_for_france['WorkMethodsFrequencyCross-Validation'].isin(['Often', 'Most of the time']))
]

probability = 100 * new_df_for_france.shape[0] / multichoice_df.shape[0]


print("Joint probability that a respondent is from France, earns less than 100,000 USD annually, \nand uses Cross-Validation Often or Most of the time = {:.2f}%".format(probability))




Joint probability that a respondent is from France, earns less than 100,000 USD annually, 
and uses Cross-Validation Often or Most of the time = 0.46%


Q2-g

In [18]:
# using Bayes Theorem P(A|B) = (P(B|A) * P(B)) / P(A)
total_programmers_df = multichoice_df[multichoice_df['CurrentJobTitleSelect'].str.contains('Programmer', na=False)]
total_prog_count = total_programmers_df.shape[0]


prob = (100 * probability_programmer_given_c_cpp * total_prog_count) / total_c_users

print("Probability that a respondent uses C/C++ at work given they are employed as a Programmer: {:.2f}%".format(prob))


Probability that a respondent uses C/C++ at work given they are employed as a Programmer: 67.28%


Q2-h

In [ ]:
wearing_glasses = 0.15
prob_wearing_glasses_given_PhD = 0.25

phd_df = multichoice_df[multichoice_df['FormalEducation'] == "Doctoral degree"]

# Calculate the probability
probability_phd = phd_df.shape[0] / multichoice_df.shape[0]

asked_prob = prob_wearing_glasses_given_PhD * probability_phd / wearing_glasses

print("Probability of a respondent having a PhD given that they wear glasses = {:.2f}%".format(asked_prob))

Probability of a respondent having a PhD given that they wear glasses = 0.23%
